In [9]:
# from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN
import pandas as pd
import scipy.sparse as sp

In [13]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [91]:
def encode_edges(edges_data):
    new_series = edges_data[0].append(edges_data[1])
    labels, levels = pd.factorize(new_series)

    edgeId = {}

    for i in range(len(levels)):
        key = levels[i]
        edgeId[key] = i

    encoded_edges = []
    for row in edges_data.itertuples(index=True, name='Pandas'):
        encoded_edges.append([edgeId[row[1]], edgeId[row[2]]])
    return np.asarray(encoded_edges)

In [153]:
def remove_empty_edges(mapped_edges):
    new_list = []
    for edge in mapped_edges:
        print(edge)
        print(mapped_edges[edge])
        if(edge[0] != None and edge[1] != None):
            new_list.append(edge)
    return edge

In [154]:
#https://github.com/tkipf/pygcn/issues/39
def load_data(dataset="embeddings.h5"):
    print('Loading {} dataset...'.format(dataset))

    #idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    idx_features_labels = pd.read_hdf(dataset, 'df')
    
    #reindex to make sure columns are in correct order
    cols = list(idx_features_labels)
    # move the column to head of list using index, pop and insert
    cols.insert(0, cols.pop(cols.index('article_name')))
    idx_features_labels = idx_features_labels.ix[:, cols]
    idx_features_labels = idx_features_labels.values
    print(idx_features_labels)
    
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.str)
    idx_map = {j: i for i, j in enumerate(idx)}
    #edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    #edges_unordered = np.genfromtxt("relations.txt", dtype=np.int32)
    edges_data = pd.read_csv("relations.txt", sep=" ", header=None)
    edges_unordered = edges_data#encode_edges(edges_data)
    mapped_edges = map(idx_map.get, edges_unordered.values.flatten())
    print(mapped_edges)
    mapped_edges = remove_empty_edges(mapped_edges)
    edges = np.array(mapped_edges, dtype=np.int32).reshape(edges_unordered.shape)
    
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [155]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [156]:
adj, features, labels, idx_train, idx_val, idx_test = load_data("embeddings.h5")

Loading embeddings.h5 dataset...
[['Machine_learning' 0.0 0.0 ... 0.0 0.0 0.0]
 ['Artificial_neural_networks' 0.0 0.0 ... 0.0 0.0 0.0]
 ['Neural_network_software' 0.0 0.0 ... 0.0 0.0 0.0]
 ...
 ['Decision_trees' 0.0 0.0 ... 0.0 0.0 0.0]
 ['Machine_learning_task' 0.0 0.0 ... 0.0 0.0 0.0]
 ['Ensemble_learning' 0.0 0.0 ... 0.0 0.0 0.0]]
57


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


TypeError: 'map' object is not subscriptable

In [152]:
import pygcn

In [4]:
pygcn

<module 'pygcn' from '/Users/curtis/Desktop/Wikipedia-Graph-Topic-Suggestion/pygcn/__init__.py'>

In [13]:
import numpy

In [14]:
numpy.version.version

'1.17.0'

In [12]:
!pip3 install numpy==1.15.4

    100% |████████████████████████████████| 24.5MB 2.1MB/s eta 0:00:01
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.4 which is incompatible.
mxnet 1.4.0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.15.4 which is incompatible.
bert-embedding 1.0.1 has requirement numpy==1.14.6, but you'll have numpy 1.15.4 which is incompatible.
  Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0
You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
